In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [7]:
# Make list of participants

folder = './2_PAT/'
files = os.listdir(folder)

IDs = []
for file in files:
    IDs.append(file.split('_')[1].split('-')[0])

uniqueIDs = list(set(IDs))
uniqueIDs.sort()
N_part = len(uniqueIDs)
print('Nb participants = ' + str(N_part))

Nb participants = 170


In [3]:
def running_mean(isHit, window_size):
    
    data_arr = np.array(isHit)

    window_trials = []
    window_values = []

    for i in range(len(data_arr)+1-window_size):

        trials_tmp = list(range(i,i+window_size))

        window_trials.append(trials_tmp)
        window_values.append(data_arr[trials_tmp])

    window_means = [np.mean(window) for window in window_values]
    
    return window_means, window_values, window_trials

In [4]:
### Figure
def plot_mean_hits_reward(ax1, ax2, ax3, cue_str, isHit, fbs, rews, vt, window_size):

    ### Subplot 1 

    # Plot means
    window_means, window_values, window_trials = running_mean(isHit, window_size)
    window_end_trial = [window[-1]+1 for window in window_trials]
    l_means = ax1.plot(window_end_trial, window_means, c='#1f77b4', linestyle='-', linewidth=1.5)

    # Plot fbs
    height_fb = 1.1
    x = [ind+1 for ind, el in enumerate(fbs) if el == 5]
    l_fb_5 = ax1.scatter(x, height_fb * np.ones(len(x)), c='green', alpha=0.9)
    x = [ind+1 for ind, el in enumerate(fbs) if el == 1]
    l_fb_1 = ax1.scatter(x, height_fb * np.ones(len(x)), c='green', alpha=0.1)
    x = [ind+1 for ind, el in enumerate(fbs) if el == -1]
    l_fb_m1 = ax1.scatter(x, height_fb * np.ones(len(x)), c='red', alpha=0.1)
    x = [ind+1 for ind, el in enumerate(fbs) if el == -5]
    l_fb_m5 = ax1.scatter(x, height_fb * np.ones(len(x)), c='red', alpha=0.9)

    # Plot hits
    height_hit = 1.15
    x_hit = [ind+1 for ind, el in enumerate(isHit) if el == 1]
    l_hits = ax1.scatter(x_hit, height_hit * np.ones(len(x_hit)), c='black', alpha=1, marker='v')

    # Labels etc 
    ax1.set_xlabel('', fontsize = 18)
    ax1.set_yticks(np.arange(0,1.2,step=0.2))
    ax1.set_ylim([-0.1,1.2])
    ax1.set_ylabel('Hit percentage', fontsize = 18)
    ax1.set_title(cue_str, fontsize = 18, fontweight='bold')

    # Legends
    ax1.legend([l_fb_5, l_fb_1, l_fb_m1, l_fb_m5, l_hits], ['FB_5', 'FB_1', 'FB_-1', 'FB_-5', 'Hit'], frameon=False)
    ax1_bis = ax1.twinx()
    ax1_bis.get_yaxis().set_visible(False)
    ax1_bis.legend(l_means, ['Running average\nwindow size = '+ str(window_size)], loc='center left', bbox_to_anchor=(0, 0.6), frameon=False)

    
    ### Subplot 2
    
    # Plot expected values
    ax2.plot(list(range(1,len(vt)+1)), vt, c='#1f77b4', linestyle='-', linewidth=1.5)
    
    # Plot horizontal line at 0
    ax2.plot(np.arange(-1,30), np.zeros(len(np.arange(-1,30))), 'k--', alpha=0.5, linewidth=1.0)
    
    # Labels etc 
    ax2.set_xlabel('', fontsize = 18)
    ax2.set_ylabel('Expected\nvalues', fontsize = 16)
    
    
    ### Subplot 3

    # Plot accumulating reward
    ax3.plot(list(range(1,len(rews)+1)), np.cumsum(rews), c='#1f77b4', linestyle='-', linewidth=1.5)

    # Plot horizontal line at 0
    ax3.plot(np.arange(-1,30), np.zeros(len(np.arange(-1,30))), 'k--', alpha=0.5, linewidth=1.0)

    # Labels etc 
    ax3.set_xlabel('Trials', fontsize = 16)
    ylim = round(max(abs(np.cumsum(rews)))+6,-1)
    ax3.set_ylim([-ylim,ylim])
    ax3.set_ylabel('Accum. reward', fontsize = 16)
    
    
    # Same characteristics for each ax
    
    for ax in [ax1,ax2,ax3]:
        
        # Plot vertical grid
        ax.grid(axis="x", alpha=0.1)
    
        # x-axis (trials)
        x_ticks = list(range(1,len(fbs)+1))
        ax.set_xlim(x_ticks[0]-1,x_ticks[-1]+1)
        ax.set_xticks(x_ticks)
        
        
    
    return ax1, ax2, ax3

In [5]:
def get_expected_values(fbs, v0, alpha):
    
    # Initialise vector of values
    vt = np.empty(len(fbs))
    vt.fill(np.nan)

    # Fill in prior
    vt[0] = v0

    # Iterate to fill in vector
    for t in range(1,len(vt)):
    
        # Compute prediction error
        pe = fbs[t-1] - vt[t-1]

        # Compute new vt and fill in 
        vt[t] = vt[t-1] + alpha * pe
    
    return vt

In [8]:
# Parameters for running average
window_size = 8

# Parameters for expected values
v0 = 0
alpha = 0.2

# Loop overs IDs
#uniqueIDs = ['007']

for ID in uniqueIDs: 
    
    print(ID)
    
    # Load df2_cf 
    user_folder = 'data/user_' + ID + '/'
    df2_cf = pd.read_pickle(user_folder + 'df2_cf.pkl')
    
    ### Figure
    fig, axs = plt.subplots(6, 2, figsize=(16,14), gridspec_kw={'height_ratios': [2, 1, 1, 2, 1, 1]}, constrained_layout=True)
    fig.patch.set_facecolor('white')

    
    ## Upper left
    cue_str = 'Cue_HR'
    cue_data = df2_cf[df2_cf['Code']==cue_str] 
    plot_mean_hits_reward(axs[0][0], axs[1][0], axs[2][0], cue_str=cue_str,
                          isHit = cue_data['isHit'].tolist(), 
                          fbs = cue_data['FBs'].tolist(), 
                          rews = cue_data['rews'].tolist(), 
                          vt = get_expected_values(cue_data['FBs'].tolist(), v0, alpha), 
                          window_size=window_size)
    
    ## Bottom left
    cue_str = 'Cue_LR'
    cue_data = df2_cf[df2_cf['Code']==cue_str]
    plot_mean_hits_reward(axs[3][0], axs[4][0], axs[5][0], cue_str=cue_str,
                          isHit = cue_data['isHit'].tolist(), 
                          fbs = cue_data['FBs'].tolist(), 
                          rews = cue_data['rews'].tolist(), 
                          vt = get_expected_values(cue_data['FBs'].tolist(), v0, alpha), 
                          window_size=window_size)
    
    ## Upper right
    cue_str = 'Cue_HP'
    cue_data = df2_cf[df2_cf['Code']==cue_str]
    plot_mean_hits_reward(axs[0][1], axs[1][1], axs[2][1], cue_str=cue_str,
                          isHit = cue_data['isHit'].tolist(), 
                          fbs = cue_data['FBs'].tolist(), 
                          rews = cue_data['rews'].tolist(), 
                          vt = get_expected_values(cue_data['FBs'].tolist(), v0, alpha), 
                          window_size=window_size)
    
    ## Bottom right
    cue_str = 'Cue_LP'
    cue_data = df2_cf[df2_cf['Code']==cue_str]
    plot_mean_hits_reward(axs[3][1], axs[4][1], axs[5][1], cue_str=cue_str,
                          isHit = cue_data['isHit'].tolist(), 
                          fbs = cue_data['FBs'].tolist(), 
                          rews = cue_data['rews'].tolist(), 
                          vt = get_expected_values(cue_data['FBs'].tolist(), v0, alpha), 
                          window_size=window_size)


    fig.suptitle('User=' + ID, fontsize = 25)
    
    ### Save
    user_folder = 'data/user_' + ID + '/'

    ## Creates user folder if does not exist
    isExist = os.path.exists(user_folder)
    if not isExist:
        os.makedirs(user_folder)

    plt.savefig(user_folder + 'hits_per_cue_run_av.png')
    plt.close() # So that they are not all displayed

001
003
006
007
008
009
010
012
014
023
028
032
033
034
035
036
038
044
045
049
050
054
055
057
058
060
072
073
075
076
077
078
080
084
085
086
088
090
093
096
100
101
103
104
106
107
108
109
112
114
115
118
119
120
121
122
130
132
133
136
137
138
142
143
145
147
149
151
155
158
159
165
170
172
173
175
176
177
178
181
185
186
187
189
199
204
207
208
211
213
217
221
229
230
236
237
239
240
244
251
252
258
259
263
269
271
280
282
283
284
286
287
291
292
293
301
302
303
306
313
314
315
319
321
322
325
326
327
329
335
336
337
339
341
345
349
351
360
361
362
368
375
376
381
384
390
391
393
395
397
400
405
406
412
413
414
421
422
423
427
437
438
440
446
450
453
462
469
470
471
